**Create CNN Model and Optimize it using Keras Tuner**

In [13]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 9.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78935 sha256=d0706038c3dd8b1bbda79f3cafbd4489a9298acd4b4c0caa0be074801394cd31
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=b6536d49b0157a409f5462cd57188275b5ecd8bc113b51b000daa3b246716a96
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 

In [4]:
print(tf.__version__)

2.5.0


In [5]:
fashion_mnist = keras.datasets.fashion_mnist

In [6]:
fashion_mnist

<module 'tensorflow.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.7/dist-packages/tensorflow/keras/datasets/fashion_mnist/__init__.py'>

In [7]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [8]:
train_images = train_images/255.0
test_images = test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1) 

In [11]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [15]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5,
                            directory='output',
                            project_name='Mnist Fashion')

In [16]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 00m 41s]
val_accuracy: 0.862333357334137

Best val_accuracy So Far: 0.9066666960716248
Total elapsed time: 00h 03m 29s
INFO:tensorflow:Oracle triggered exit


In [17]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,377,194
Trainable params: 1,377,194
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0277 - accuracy: 0.9904 - val_loss: 0.5519 - val_accuracy: 0.9120
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0251 - accuracy: 0.9909 - val_loss: 0.5478 - val_accuracy: 0.9097
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0207 - accuracy: 0.9928 - val_loss: 0.6114 - val_accuracy: 0.9137
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0205 - accuracy: 0.9930 - val_loss: 0.6167 - val_accuracy: 0.9115
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0166 - accuracy: 0.9945 - val_loss: 0.6749 - val_accuracy: 0.9058
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0173 - accuracy: 0.9942 - val_loss: 0.6579 - val_accuracy: 0.9120
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0181 - accuracy: 0.9943 - val_loss: 0.7164 - val_accuracy

In [23]:
 !git clone https://github.com/mdeshkar/mnist.git

Cloning into 'mnist'...


In [24]:
!git status

fatal: not a git repository (or any of the parent directories): .git
